In [1]:
from __future__ import print_function

import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
from six.moves import range
import MeCab
import cPickle as pickle

import app.common.utils as utils
from app.models.sequence import Sequence
from app.tensorflow.batch_generator import BatchGenerator
from app.tensorflow.train import model
import app.common.settings as settings

mecab = utils.mecab

In [2]:
print(mecab.parse('おはよう御座います'))

おはよう 御座い ます 



In [3]:
print(mecab.parse('おはようございます'))

おはよう ござい ます 



In [4]:
print(mecab.parse('御座る'))

御座る 



In [5]:
print(mecab.parse('動く凱はお初じゃない？'))
words = mecab.parse('動く凱はお初じゃない？').split()
for w in words:
  print(w)

動く 凱 は お初 じゃ ない ？ 

動く
凱
は
お初
じゃ
ない
？


In [ ]:
graph = tf.Graph()
inputs, labels, sequence_lengths, optimizer, loss, predictions, summary_op, saver = model(graph)
train_writer = tf.train.SummaryWriter('tensorboard/train', graph)
test_writer = tf.train.SummaryWriter('tensorboard/test', graph)
with tf.Session(graph=graph) as sess:
    sess.run(tf.initialize_all_variables())
    print("Training...")
    train_batches = BatchGenerator(settings.BATCH_SIZE)
    test_batches = BatchGenerator(3000, settings.SEQUENCE_LIMIT)
    test_inputs, test_labels, test_sequence_lengths = test_batches.next()

    for step in xrange(settings.EPOCH):
        if step % 1000 != 1:
            train_inputs, train_labels, train_sequence_lengths = train_batches.next()
            feed_dict = { inputs: train_inputs, labels: train_labels, sequence_lengths: train_sequence_lengths }
            _, train_loss, train_predictions, summary = sess.run([optimizer, loss, predictions, summary_op], feed_dict=feed_dict)
            train_writer.add_summary(summary, step)
            train_writer.flush()
        else:
            test_feed_dict = { inputs: test_inputs, labels: test_labels, sequence_lengths: test_sequence_lengths }
            test_loss, summary = sess.run([loss, summary_op], feed_dict=test_feed_dict)
            test_writer.add_summary(summary, step)
            test_writer.flush()

        if step % 1000 == 0:
            print('-----------Step %d:-------------' % step)
            print('Training set:')
            print('  Loss       : ', train_loss)
            print('  Input      : ', train_inputs[0])
            print('  Label      : ', utils.onehot2sticker(train_labels))
            print('  Prediction : ', utils.onehot2sticker(train_predictions))

Training...
-----------Step 0:-------------
Training set:
  Loss       :  3.82859
  Input      :  [6671, 1968, 8746, 3, 4985, 0, 0, 0, 0, 0]
  Label      :  524
  Prediction :  26
-----------Step 1000:-------------
Training set:
  Loss       :  1.66472
  Input      :  [336, 6268, 181, 101, 11, 0, 0, 0, 0, 0]
  Label      :  157
  Prediction :  36
-----------Step 2000:-------------
Training set:
  Loss       :  1.79741
  Input      :  [43, 10, 0, 0, 0, 0, 0, 0, 0, 0]
  Label      :  179
  Prediction :  36
-----------Step 3000:-------------
Training set:
  Loss       :  1.87234
  Input      :  [42, 39, 0, 0, 0, 0, 0, 0, 0, 0]
  Label      :  172
  Prediction :  172
-----------Step 4000:-------------
Training set:
  Loss       :  2.24367
  Input      :  [1632, 1732, 1632, 0, 0, 0, 0, 0, 0, 0]
  Label      :  144
  Prediction :  144
-----------Step 5000:-------------
Training set:
  Loss       :  1.94571
  Input      :  [276, 294, 103, 255, 0, 0, 0, 0, 0, 0]
  Label      :  36
  Prediction